In [23]:
import re
import requests
import os
import json
from pprint import pprint

In [3]:
header_raw = """
Host: dblp.uni-trier.de
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:123.0) Gecko/20100101 Firefox/123.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8
Accept-Language: zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2
Accept-Encoding: gzip, deflate, br
DNT: 1
Sec-GPC: 1
Connection: keep-alive
Cookie: dblp-search-mode=c; dblp-dismiss-new-feature-2022-01-27=1
Upgrade-Insecure-Requests: 1
Sec-Fetch-Dest: document
Sec-Fetch-Mode: navigate
Sec-Fetch-Site: cross-site
"""

header = {}
for line in header_raw.split('\n'):
    if line.strip() == '':
        continue
    k, v = line.split(': ')
    header[k] = v

print(header)

{'Host': 'dblp.uni-trier.de', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:123.0) Gecko/20100101 Firefox/123.0', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8', 'Accept-Language': 'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2', 'Accept-Encoding': 'gzip, deflate, br', 'DNT': '1', 'Sec-GPC': '1', 'Connection': 'keep-alive', 'Cookie': 'dblp-search-mode=c; dblp-dismiss-new-feature-2022-01-27=1', 'Upgrade-Insecure-Requests': '1', 'Sec-Fetch-Dest': 'document', 'Sec-Fetch-Mode': 'navigate', 'Sec-Fetch-Site': 'cross-site'}


In [4]:
# open https://dblp.uni-trier.de/db/conf/kdd/kdd2023.html and save to page.txt
req = requests.get('https://dblp.uni-trier.de/db/conf/kdd/kdd2023.html', headers=header)
with open('page.txt', 'w') as f:
    f.write(req.text)

In [15]:
with open('page.txt', 'r') as f:
    page = f.read()

In [29]:
class Paper:
    author: list[str]
    title: str
    startPage: int
    endPage: int

class Track:
    track: str
    papers: list[Paper]

all_papers: list[(int, Paper)] = []
all_tracks: list[Track] = []

In [32]:
# regex patterns:

track_full_pattern = re.compile(r"""<header class="h2"><h2 id="secResearchTrackFullPapers">Research Track Full Papers</h2></header>""")

# <header class="h2"><h2 id="secResearchTrackFullPapers">Research Track Full Papers</h2></header>
track_name_pattern = re.compile(r"""<header class="h2"><h2 id="(.+?)">(.+?)</h2></header>""")

# <span itemprop="author" itemscope="" itemtype="http://schema.org/Person"><a href="https://dblp.uni-trier.de/pid/211/5760.html" itemprop="url"><span itemprop="name" title="Florian Adriaens">Florian Adriaens</span></a><img src="https://dblp.uni-trier.de/img/orcid-mark.12x12.png" style="padding-left:0.25em;" alt="" title="0000-0001-7820-6883"></span>
author_name_pattern = re.compile(r"""<span itemprop="author" itemscope="" itemtype="http://schema.org/Person"><a href="https://dblp.uni-trier.de/pid/.+?\.html" itemprop="url"><span itemprop="name" title="(.+?)">.+?</span></a><img src="https://dblp.uni-trier.de/img/orcid-mark.12x12.png" style="padding-left:0.25em;" alt="" title=".+?"></span>""")

# <span class="title" itemprop="name">Minimizing Hitting Time between Disparate Groups with Shortcut Edges.</span>
title_pattern = re.compile(r"""<span class="title" itemprop="name">(.+?)</span>""")

# <span itemprop="pagination">1-10</span>
pagination_pattern = re.compile(r"""<span itemprop="pagination">(\d+)-(\d+)</span>""")

In [ ]:
# use track_name_pattern to find all tracks, then use other patterns to find papers in each track
tracks = track_name_pattern.findall(page)
for track_id, track_name in tracks:
    print(track_id, track_name)
    track = Track()
    track.track = track_name
    track.papers = []
    all_tracks.append(track)

    # find all papers in this track
    papers = re.findall(f"""<h3 class="title" itemprop="name"><a href="https://dblp.uni-trier.de/db/conf/kdd/kdd2023.html#(.+?)">.+?</a></h3>""", page)
    for paper_id in papers:
        paper = Paper()
        paper.author = author_name_pattern.findall(page)
        paper.title = title_pattern.findall(page)
        paper.startPage, paper.endPage = map(int, pagination_pattern.findall(page)[0])
        all_papers.append((track_id, paper))